In [1]:
# 4_recommendation_algorithm.ipynb

# Recommendation Algorithm for KuaiRec Recommender System
# ======================================================
#
# This notebook covers the fourth task of the project:
# 1. Loading the trained models
# 2. Implementing the recommendation algorithm
# 3. Generating recommendations for all users in the test set
# 4. Evaluating recommendation quality

# Import necessary libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Add the src directory to the path so we can import our modules
sys.path.append(os.path.abspath("../"))
from src.recommender import KuaiRecRecommender

# Set up directories
processed_dir = "../data/processed"
models_dir = "../models"
output_dir = "../results"
os.makedirs(output_dir, exist_ok=True)

# Load test data
print("Loading test data...")
test_features = pd.read_csv(os.path.join(processed_dir, "test_features.csv"), low_memory=True)
print(f"Test features shape: {test_features.shape}")

# Initialize the recommender
print("Initializing recommender...")
recommender = KuaiRecRecommender(models_dir=models_dir, processed_dir=processed_dir)

# Generate recommendations for a few example users
print("\n--- Example Recommendations ---")
example_users = test_features['user_id'].unique()[:5]  # Use the first 5 users as examples

for user_id in example_users:
    print(f"\nRecommendations for user {user_id}:")
    
    # Get recommendations with default weights
    recs = recommender.recommend(user_id, n=5)
    
    # Display recommendations
    if recs:
        for rank, (item_id, score) in enumerate(recs):
            print(f"  {rank+1}. Video {item_id} (score: {score:.4f})")
    else:
        print("  No recommendations generated.")

# Generate recommendations for all users in the test set
print("\n--- Generating Recommendations for All Test Users ---")
test_users = test_features['user_id'].unique()
all_recommendations = recommender.generate_recommendations_for_all_users(
    users=test_users, 
    n=10,
    weights={
        'collaborative': 0.4,
        'content': 0.3,
        'sequence': 0.2,
        'hybrid': 0.1
    }
)

# Save recommendations to file
recommender.save_recommendations(
    all_recommendations, 
    os.path.join(output_dir, "recommendations.csv")
)

# Summary statistics
print("\n=== Recommendation Algorithm Summary ===")
print(f"1. Number of users with recommendations: {len(all_recommendations)}")
print(f"2. Average number of recommendations per user: {np.mean([len(recs) for recs in all_recommendations.values()]):.2f}")
print(f"3. All recommendations have been saved to {os.path.join(output_dir, 'recommendations.csv')}")
print("\nNext step: Evaluate the recommendations using appropriate metrics.")

Loading test data...
Test features shape: (934735, 21)
Initializing recommender...
Loaded interaction matrix.
Loaded collaborative filtering model.
Loaded content-based model.
Loaded sequence-aware model.
Loaded hybrid model.

--- Example Recommendations ---

Recommendations for user 14:
  1. Video 314 (score: 1.8789)
  2. Video 600 (score: 1.8765)
  3. Video 1305 (score: 1.8115)
  4. Video 4123 (score: 1.6332)
  5. Video 9815 (score: 1.4728)

Recommendations for user 19:
  1. Video 4760 (score: 7.3000)
  2. Video 3136 (score: 5.7000)
  3. Video 3138 (score: 4.4000)
  4. Video 6184 (score: 4.2000)
  5. Video 1429 (score: 3.7000)

Recommendations for user 21:
  1. Video 6204 (score: 2.6000)
  2. Video 600 (score: 1.8752)
  3. Video 2130 (score: 1.8199)
  4. Video 4123 (score: 1.7542)
  5. Video 6787 (score: 1.6938)

Recommendations for user 23:
  1. Video 5711 (score: 2.2000)
  2. Video 8340 (score: 1.6363)
  3. Video 845 (score: 1.5000)
  4. Video 6250 (score: 1.3000)
  5. Video 1445 (

Generating recommendations: 100%|██████████| 1411/1411 [01:03<00:00, 22.17it/s]

Saved recommendations to ../results\recommendations.csv

=== Recommendation Algorithm Summary ===
1. Number of users with recommendations: 1411
2. Average number of recommendations per user: 10.00
3. All recommendations have been saved to ../results\recommendations.csv

Next step: Evaluate the recommendations using appropriate metrics.
